In [1]:
from collections import namedtuple
import os
os.getcwd()
os.chdir("../")
os.getcwd()

'c:\\Users\\arunk\\ComputerVision\\CNN\\DEEPCNNClassifier'

In [2]:
DataIngestionConfig =namedtuple("DataIngestionConfig",[
    'root_dir',
    "source_URL",
    "local_data_file",
    "unzip_dir"])

In [ ]:
from dataclasses import dataclass

from pathlib import Path

@dataclass(forzen=True)
class DataIngestionConfig:
    root_dir:Path
    source_URL = str
    local_data_file =Path
    unzip_dir = Path

In [3]:
import sys
sys.path.append(r"C:\Users\arunk\ComputerVision\CNN\DEEPCNNClassifier\src\deepClassifier")

In [4]:
from distutils.command.config import config
from deepClassifier.utils import read_yaml,create_directories
from deepClassifier.constants import PARAMS_FILE_PATH,CONFIG_FILE_PATH
from deepClassifier.entity import DataIngestionConfig
from deepClassifier.config import ConfigurationManager

In [4]:
from deepClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
import urllib.request as request

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config = self.config.dataingestion
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig( 
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [13]:
import os
import urllib.request
from zipfile import ZipFile

In [16]:
from urllib import request
from deepClassifier.components.data_Ingestion import DataIngestion


class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def downloadfile(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
             )

    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]
    
    def _preprocess(self, zf: ZipFile, f:str, working_dir:str):
        target_filepath =os.path.join(working_dir,f)
        if not os.path.exists(target_filepath):
            zf.extract(f,working_dir)
            
        if os.path.getsize(target_filepath)==0:
            os.remove(target_filepath)

    def unzip_and_clean(self):
        with ZipFile(file= self.config.local_data_file, mode='r') as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf,f,self.config.unzip_dir)

In [17]:
from deepClassifier.config import ConfigurationManager

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.downloadfile()
    data_ingestion.unzip_and_clean()
except Exception as e:
        raise e

(2022-09-18 18:49:05,601:INFO:common: yaml file: configs\config.yaml loaded successfully)
(2022-09-18 18:49:05,603:INFO:common: yaml file: params.yaml loaded successfully)
(2022-09-18 18:49:05,605:INFO:common: created directory at: artifacts)
(2022-09-18 18:49:05,606:INFO:common: created directory at: artifacts/data_ingestion)
